## Обучение RL агента

Релизация из CleanRL https://docs.cleanrl.dev/rl-algorithms/td3/ ,
я изменил код чтобы работало для PointMaze

Среда PointMaze со spars reward, то есть 1 если цель достигнута, 0 иначе

Где я изменил я добавил комментарий # Исправлено, и закомментил оригинальый код

In [3]:
# Релизация из CleanRL, я изменил код чтобы работало для PointMaze
import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import gymnasium_robotics
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tyro
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


# Наш П-образный лабиринт
# буква 'с' обозначает место инициализации агента и цели, каждый эпизод рандомно
example_map = [[1, 1, 1, 1, 1],
        [1, 'c', 0, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 'c', 0, 0, 1],
        [1, 1, 1, 1, 1]]

/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchPickAndPlace-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-pack

In [4]:
# Параметры кода
@dataclass
class Args:
    # exp_name: str = os.path.basename(__file__)[: -len(".py")]
    exp_name: str = os.path.basename(__file__)[: -len(".py")] if "__file__" in locals() else "ipynb_experiment"
    """the name of this experiment"""
    seed: int = 1
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = True
    """if toggled, cuda will be enabled by default"""
    track: bool = False            #Логируем в wandb
    """if toggled, this experiment will be tracked with Weights and Biases"""
    wandb_project_name: str = "AIRI_RL"    #Название проекта в wandb
    """the wandb's project name"""
    wandb_entity: str = None
    """the entity (team) of wandb's project"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""
    save_model: bool = True   #Сохраняем модель
    """whether to save model into the `runs/{run_name}` folder"""
    upload_model: bool = False
    """whether to upload the saved model to huggingface"""
    hf_entity: str = ""
    """the user or org name of the model repository from the Hugging Face Hub"""

    # Algorithm specific arguments
    # env_id: str = 'PointMaze_UMazeDense-v3'  #Наша среда лабиринта c непрерывным reward (евклид расстояние)
    env_id: str = 'PointMaze_UMaze-v3' #Наша среда лабиринта cо spars rewand (1 при достижении цели)
    """the id of the environment"""
    total_timesteps: int = 30000   #Изменяем суммарное количесво шагов агента
    """total timesteps of the experiments"""
    learning_rate: float = 3e-4
    """the learning rate of the optimizer"""
    num_envs: int = 1           # Можно увеличить, но пока не знаю как повлияет, надо поменять код чтобы не смломалось при значениях больших 1
    """the number of parallel game environments"""
    buffer_size: int = int(1e6)
    """the replay memory buffer size"""
    gamma: float = 0.99
    """the discount factor gamma"""
    tau: float = 0.005
    """target smoothing coefficient (default: 0.005)"""
    batch_size: int = 256
    """the batch size of sample from the reply memory"""
    policy_noise: float = 0.2
    """the scale of policy noise"""
    exploration_noise: float = 0.1
    """the scale of exploration noise"""
    learning_starts: int = 25e3
    """timestep to start learning"""
    policy_frequency: int = 2
    """the frequency of training policy (delayed)"""
    noise_clip: float = 0.5
    """noise clip parameter of the Target Policy Smoothing Regularization"""

In [5]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array", maze_map=example_map)  #Создаем среду, можем добавить сложный лабиринт maze_map=example_map
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)
        return env

    return thunk

# Многие строки изначальной реализации изменены из-за рызличий gym и gym_robotics, у них разные атрибуты классов (вроде)
# Код TD3 написан для сред из GYM, я изменил для pointmase из gym_robotics
# Исправленные строки обочзначены "# Исправлено"

# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        # self.fc1 = nn.Linear(
        #     np.array(env.single_observation_space.shape).prod() + np.prod(env.single_action_space.shape),
        #     256,
        # )
        #  Иправлено
        # self.fc1 = nn.Linear(
        #     env.observation_space['observation'].shape[0] + env.observation_space['desired_goal'].shape[0] + env.action_space.shape[0],
        #     256,
        # )
        self.fc1 = nn.Linear(
            8, 
            256,
        )


        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x, a):
        x = torch.cat([x, a], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    


class Actor(nn.Module):
    def __init__(self, env):
        super().__init__()
        # Исправлено
        # self.fc1 = nn.Linear(np.array(env.single_observation_space.shape).prod(), 256)
        # self.fc1 = nn.Linear(env.observation_space['observation'].shape[0] + env.observation_space['desired_goal'].shape[0], 256)
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 256)
        # self.fc_mu = nn.Linear(256, np.prod(env.single_action_space.shape))
        self.fc_mu = nn.Linear(256, env.action_space.shape[1])
        # action rescaling

        # Поменять? env.single_action_space.high
        # Исправлено
        self.register_buffer(
            "action_scale",
            torch.tensor(
                # (env.single_action_space.high - env.single_action_space.low) / 2.0,
                (env.action_space.high - env.action_space.low) / 2.0,
                dtype=torch.float32,
            ),
        )
        self.register_buffer(
            "action_bias",
            torch.tensor(
                # (env.single_action_space.high + env.single_action_space.low) / 2.0,
                (env.action_space.high + env.action_space.low) / 2.0,
                dtype=torch.float32,
            ),
        )

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc_mu(x))
        return x * self.action_scale + self.action_bias

In [6]:
import stable_baselines3 as sb3

if sb3.__version__ < "2.0":
        raise ValueError(
            """Ongoing migration: run the following command to install the new dependencies:
poetry run pip install "stable_baselines3==2.0.0a1"
"""
        )

# Исправлено
# args = tyro.cli(Args)
args = Args()
run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"

In [7]:
if args.track:
    import wandb

    wandb.init(
            project=args.wandb_project_name,
            entity=args.wandb_entity,
            sync_tensorboard=True,
            config=vars(args),
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

In [8]:
# TRY NOT TO MODIFY: seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

In [9]:
# env setup
envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, args.seed + i, i, args.capture_video, run_name) for i in range(args.num_envs)]
    )
assert isinstance(envs.single_action_space, gym.spaces.Box), "only continuous action space is supported"

In [10]:
actor = Actor(envs).to(device)
qf1 = QNetwork(envs).to(device)
qf2 = QNetwork(envs).to(device)
qf1_target = QNetwork(envs).to(device)
qf2_target = QNetwork(envs).to(device)
target_actor = Actor(envs).to(device)
target_actor.load_state_dict(actor.state_dict())
qf1_target.load_state_dict(qf1.state_dict())
qf2_target.load_state_dict(qf2.state_dict())
q_optimizer = optim.Adam(list(qf1.parameters()) + list(qf2.parameters()), lr=args.learning_rate)
actor_optimizer = optim.Adam(list(actor.parameters()), lr=args.learning_rate)

In [11]:
# Исправлено
# envs.single_observation_space.dtype = np.float32
envs.single_observation_space['observation'].dtype = np.float32
envs.single_observation_space['desired_goal'].dtype = np.float32

# Добавлено
from gymnasium.spaces import Box
observation_space = Box(
    low=-np.inf,
    high=np.inf,
    shape=(6,),  # Размерность 2 + 4 = 6
    dtype=np.float32
)

rb = ReplayBuffer(
    args.buffer_size,
    # envs.single_observation_space,
    # envs.single_observation_space['observation'],
    observation_space, 
    envs.single_action_space,
    device,
    n_envs=args.num_envs,
    handle_timeout_termination=False,
)

In [12]:
#   Тут можете изменять количество итераций
#   тут n_inerations итераций по 30к эпизодов, сделал так, так как обучение постепенно замедляется по времени почему-то

n_inerations = 2

for i in tqdm(range(n_inerations)): 
    start_time = time.time()

    # TRY NOT TO MODIFY: start the game
    obs, _ = envs.reset(seed=args.seed)
    for global_step in (range(args.total_timesteps)):
        # ALGO LOGIC: put action logic here
        if global_step < args.learning_starts:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            with torch.no_grad():
                # Исправлено
                # actions = actor(torch.Tensor(obs['observation']).to(device))
                # actions = actor(torch.Tensor(np.concatenate([obs['observation'][0],obs['desired_goal'][0]], dtype=np.float32)).to(device))
                observation_part = obs['observation'][0]  # shape (4,)
                goal_part = obs['desired_goal'][0]        # shape (2,)
                flattened_obs = np.concatenate([observation_part, goal_part])
                input_tensor = torch.FloatTensor(flattened_obs).unsqueeze(0).to(device)
                actions = actor(input_tensor[0])




                actions += torch.normal(0, actor.action_scale * args.exploration_noise)
                actions = actions.cpu().numpy().clip(envs.single_action_space.low, envs.single_action_space.high)

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, rewards, terminations, truncations, infos = envs.step(actions)

        # TRY NOT TO MODIFY: record rewards for plotting purposes
        if "final_info" in infos:
            for info in infos["final_info"]:
                if info is not None:
                    print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                    writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                    writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
                    break

        # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
        real_next_obs = next_obs.copy()
        # Под вопросом???
        # for idx, trunc in enumerate(truncations):
        #     if trunc:
        #         real_next_obs[idx] = infos["final_observation"][idx]
        
        # Исправлено
        # rb.add(obs, real_next_obs, actions, rewards, terminations, infos)
        rb.add(np.concatenate([obs['observation'][0],obs['desired_goal'][0]], dtype=np.float32),
                np.concatenate([real_next_obs['observation'][0],real_next_obs['desired_goal'][0]], dtype=np.float32), actions, rewards, terminations, infos)
        

        # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
        obs = next_obs

        # ALGO LOGIC: training.
        if global_step > args.learning_starts:
            data = rb.sample(args.batch_size)
            with torch.no_grad():
                clipped_noise = (torch.randn_like(data.actions, device=device) * args.policy_noise).clamp(
                    -args.noise_clip, args.noise_clip
                ) * target_actor.action_scale

                next_state_actions = (target_actor(data.next_observations) + clipped_noise).clamp(
                    envs.single_action_space.low[0], envs.single_action_space.high[0]
                )
                qf1_next_target = qf1_target(data.next_observations, next_state_actions)
                qf2_next_target = qf2_target(data.next_observations, next_state_actions)
                min_qf_next_target = torch.min(qf1_next_target, qf2_next_target)
                next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * args.gamma * (min_qf_next_target).view(-1)

            qf1_a_values = qf1(data.observations, data.actions).view(-1)
            qf2_a_values = qf2(data.observations, data.actions).view(-1)
            qf1_loss = F.mse_loss(qf1_a_values, next_q_value)
            qf2_loss = F.mse_loss(qf2_a_values, next_q_value)
            qf_loss = qf1_loss + qf2_loss

            # optimize the model
            q_optimizer.zero_grad()
            qf_loss.backward()
            q_optimizer.step()

            if global_step % args.policy_frequency == 0:
                actor_loss = -qf1(data.observations, actor(data.observations)).mean()
                actor_optimizer.zero_grad()
                actor_loss.backward()
                actor_optimizer.step()

                # update the target network
                for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
                for param, target_param in zip(qf1.parameters(), qf1_target.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
                for param, target_param in zip(qf2.parameters(), qf2_target.parameters()):
                    target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)

            if global_step % 100 == 0:
                writer.add_scalar("losses/qf1_values", qf1_a_values.mean().item(), global_step)
                writer.add_scalar("losses/qf2_values", qf2_a_values.mean().item(), global_step)
                writer.add_scalar("losses/qf1_loss", qf1_loss.item(), global_step)
                writer.add_scalar("losses/qf2_loss", qf2_loss.item(), global_step)
                writer.add_scalar("losses/qf_loss", qf_loss.item() / 2.0, global_step)
                writer.add_scalar("losses/actor_loss", actor_loss.item(), global_step)
                print("SPS:", int(global_step / (time.time() - start_time)))
                writer.add_scalar(
                    "charts/SPS",
                    int(global_step / (time.time() - start_time)),
                    global_step,
                )

  0%|          | 0/2 [00:00<?, ?it/s]/home/user7/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be <class 'numpy.float32'>, actual type: float64
  logger.warn(
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be <class 'numpy.float32'>, actual type: float64
  logger.warn(
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not

global_step=299, episodic_return=[0.]
global_step=599, episodic_return=[23.]
global_step=899, episodic_return=[0.]


global_step=1199, episodic_return=[0.]
global_step=1499, episodic_return=[25.]
global_step=1799, episodic_return=[85.]
global_step=2099, episodic_return=[45.]
global_step=2399, episodic_return=[0.]
global_step=2699, episodic_return=[0.]
global_step=2999, episodic_return=[0.]
global_step=3299, episodic_return=[0.]
global_step=3599, episodic_return=[34.]
global_step=3899, episodic_return=[16.]
global_step=4199, episodic_return=[86.]
global_step=4499, episodic_return=[0.]
global_step=4799, episodic_return=[2.]
global_step=5099, episodic_return=[0.]
global_step=5399, episodic_return=[0.]
global_step=5699, episodic_return=[0.]
global_step=5999, episodic_return=[0.]
global_step=6299, episodic_return=[0.]
global_step=6599, episodic_return=[0.]
global_step=6899, episodic_return=[0.]
global_step=7199, episodic_return=[0.]
global_step=7499, episodic_return=[0.]
global_step=7799, episodic_return=[28.]
global_step=8099, episodic_return=[73.]
global_step=8399, episodic_return=[0.]
global_step=8699,

 50%|█████     | 1/2 [00:31<00:31, 31.76s/it]

global_step=29999, episodic_return=[0.]
global_step=299, episodic_return=[101.]
global_step=599, episodic_return=[0.]
global_step=899, episodic_return=[0.]
global_step=1199, episodic_return=[0.]
global_step=1499, episodic_return=[161.]
global_step=1799, episodic_return=[0.]
global_step=2099, episodic_return=[0.]
global_step=2399, episodic_return=[24.]
global_step=2699, episodic_return=[0.]
global_step=2999, episodic_return=[64.]
global_step=3299, episodic_return=[0.]
global_step=3599, episodic_return=[0.]
global_step=3899, episodic_return=[0.]
global_step=4199, episodic_return=[0.]
global_step=4499, episodic_return=[132.]
global_step=4799, episodic_return=[0.]
global_step=5099, episodic_return=[0.]
global_step=5399, episodic_return=[38.]
global_step=5699, episodic_return=[49.]
global_step=5999, episodic_return=[0.]
global_step=6299, episodic_return=[0.]
global_step=6599, episodic_return=[0.]
global_step=6899, episodic_return=[0.]
global_step=7199, episodic_return=[0.]
global_step=7499,

100%|██████████| 2/2 [01:03<00:00, 31.64s/it]

global_step=29999, episodic_return=[221.]


In [13]:
# Сохранение модели на huggingface_hub
from huggingface_hub import login

login(token = '')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


Token is valid.
Your token has been saved to /home/user7/.huggingface/token
Login successful


In [ ]:
# Сохранение модели на huggingface_hub, нужно раскомментировать

# from huggingface_hub import upload_file

# if args.save_model:

#     model_path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"
#     torch.save((actor.state_dict(), qf1.state_dict(), qf2.state_dict()), model_path)
#     print(f"model saved to {model_path}")

# upload_file(
#     path_or_fileobj=model_path,
#     path_in_repo="full_model_weights.pth",
#     repo_id="Ulandaev/td3_sparse_airi_rl",
#     repo_type="model"
#     )


## Обучение функции расстояния

In [22]:
s, s_prime, g, done = rb

TypeError: cannot unpack non-iterable ReplayBuffer object

In [24]:
rb.observations

array([[-0.09402797, -1.0361911 ,  0.00563097,  0.21456832,  1.2252319 ,
        -0.17792019]], dtype=float32)

In [29]:
np.concatenate([obs['observation'][0],obs['desired_goal'][0]])

array([1.09680782, 0.01929723, 0.        , 0.        , 0.91341494,
       1.10851068])

In [30]:
rb.next_observations

array([[[-0.09402797, -1.0361911 ,  0.00563097,  0.21456832,
          1.2252319 , -0.17792019]],

       [[-0.09566676, -1.0319135 , -0.16387907,  0.42776144,
          1.2252319 , -0.17792019]],

       [[-0.09819794, -1.0280113 , -0.25311866,  0.39022094,
          1.2252319 , -0.17792019]],

       ...,

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]]], shape=(1000000, 1, 6), dtype=float32)

In [20]:
rb.observations[1][0][0:2, 3:4]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random


class DistanceNetwork(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=256):
        super(DistanceNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, s, g):
        x = torch.cat([s, g], dim=-1)  # Объединяем s и g в один вектор
        return self.net(x)

# class ReplayBuffer:
#     def __init__(self, capacity):
#         self.buffer = deque(maxlen=capacity)
    
#     def add(self, s, s_prime, g, done):
#         self.buffer.append((s, s_prime, g, done))
    
#     def sample(self, batch_size):
#         batch = random.sample(self.buffer, min(len(self.buffer), batch_size))
#         s, s_prime, g, done = zip(*batch)
#         return (
#             torch.FloatTensor(np.array(s)),
#             torch.FloatTensor(np.array(s_prime)),
#             torch.FloatTensor(np.array(g)),
#             torch.FloatTensor(np.array(done))
    
#     def __len__(self):
#         return len(self.buffer)


class DistanceLearner:
    def __init__(self, lr=1e-3, tau=0.01):
        self.model = DistanceNetwork()  # Основная сеть
        self.target_model = DistanceNetwork()  # Целевая сеть
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.tau = tau
        self.loss_fn = nn.MSELoss()
    
    def update_target_network(self):
        # Мягкое обновление целевой сети
        for target_param, param in zip(self.target_model.parameters(), 
                                     self.model.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
    
    def compute_loss(self, batch):
        # s, s_prime, g, done = batch

        s = batch.observation
        s_prime = batch.next_observations

        #################################


        
        # Вычисляем целевые значения
        with torch.no_grad():
            target_dist = self.target_model(s_prime, g)
            y = torch.zeros_like(target_dist)
            y[~done] = 1 + target_dist[~done]
        
        # Предсказания основной сети
        pred_dist = self.model(s, g)
        
        # MSE loss
        loss = self.loss_fn(pred_dist, y)
        return loss
    
    def train_step(self, batch):
        self.optimizer.zero_grad()
        loss = self.compute_loss(batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
        self.optimizer.step()
        return loss.item()


if __name__ == "__main__":
    BUFFER_SIZE = 100000
    BATCH_SIZE = 128
    TOTAL_STEPS = 100000
    TRAIN_INTERVAL = 4
    TARGET_UPDATE_INTERVAL = 100
    
    # Инициализация
    buffer = ReplayBuffer(BUFFER_SIZE)

    learner = DistanceLearner(lr=3e-4)
    
    # Обучение
    for step in range(TOTAL_STEPS):
        # Генерация данных (в реальном RL это будет опыт агента)
        # traj = generate_trajectory()
        # for s, s_prime, g, done in traj:
        #     buffer.add(s, s_prime, g, done)
        
        # Обучение
        if step % TRAIN_INTERVAL == 0 and len(buffer) > BATCH_SIZE:
            batch = buffer.sample(BATCH_SIZE)
            loss = learner.train_step(batch)
            
            if step % 1000 == 0:
                print(f"Step {step}, Loss: {loss:.4f}")
        
        # Обновление целевой сети
        if step % TARGET_UPDATE_INTERVAL == 0:
            learner.update_target_network()
    
    # # Тестирование
    # test_s = torch.FloatTensor([[1.0, 2.0]])
    # test_g = torch.FloatTensor([[3.0, 4.0]])
    # distance = learner.model(test_s, test_g)
    # print(f"\nPredicted distance between {test_s} and {test_g}: {distance.item():.2f}")